In [1]:
import numpy as np
import time
import copy
import sys
sys.path.append('/Users/ziniuwu/Desktop/research/BayesCard')
import pandas as pd
import time
import bz2
import pickle
import logging
import ast

from DataPrepare.join_data_preparation import JoinDataPreparator
from Models.pgmpy_BN import Pgmpy_BN
from Evaluation.utils import parse_query
from Evaluation.cardinality_estimation import parse_query_single_table

In [ ]:
query = "SELECT COUNT(*) FROM climate WHERE iAvail = 0 AND iClass = 0 AND iKorean >= 0 AND iKorean <= 1 AND iMay75880 = 0 AND iRagechld >= 0 AND iRagechld <= 4 AND iRrelchld = 0"
tab_query = parse_query_single_table(query, BN_J)


In [ ]:
tab_query

In [ ]:
import time
tic = time.time()
print(BN_J.query(tab_query))
print(time.time()-tic)

In [ ]:
df = pd.read_csv("/home/ziniu.wzn/Census/Census.csv", header=0, sep=",")
df = df.drop("caseid", axis=1)
df = df.dropna(axis=0)
print(len(df))
print(df.columns)

In [ ]:
BN = Pgmpy_BN("census")
BN.build_from_data(df, algorithm="greedy", max_parents=2, ignore_cols=['id'], sample_size=10000)

In [2]:
import itertools
import networkx as nx
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from itertools import chain
import sys
sys.path.append('/Users/ziniuwu/Desktop/research/BayesCard')
from Pgmpy.factors import factor_product
from Pgmpy.models import BayesianModel, JunctionTree
from Pgmpy.inference.EliminationOrder import WeightedMinFill, MinNeighbors, MinFill, MinWeight
from Pgmpy.factors.discrete import TabularCPD

class VariableElimination(object):
    def __init__(self, model):
        self.model = model
        model.check_model()

        if isinstance(model, JunctionTree):
            self.variables = set(chain(*model.nodes()))
        else:
            self.variables = model.nodes()

        self.cardinality = {}
        self.factors = defaultdict(list)

        if isinstance(model, BayesianModel):
            self.state_names_map = {}
            for node in model.nodes():
                cpd = model.get_cpds(node)
                if isinstance(cpd, TabularCPD):
                    self.cardinality[node] = cpd.variable_card
                    cpd = cpd.to_factor()
                for var in cpd.scope():
                    self.factors[var].append(cpd)
                self.state_names_map.update(cpd.no_to_name)

        elif isinstance(model, JunctionTree):
            self.cardinality = model.get_cardinality()

            for factor in model.get_factors():
                for var in factor.variables:
                    self.factors[var].append(factor)
    
    def _get_working_factors(self, evidence):
        """
        Uses the evidence given to the query methods to modify the factors before running
        the variable elimination algorithm.
        Parameters
        ----------
        evidence: dict
            Dict of the form {variable: state}
        Returns
        -------
        dict: Modified working factors.
        """

        working_factors = {
            node: {(factor, None) for factor in self.factors[node]}
            for node in self.factors
        }

        # Dealing with evidence. Reducing factors over it before VE is run.
        if evidence:
            for evidence_var in evidence:
                for factor, origin in working_factors[evidence_var]:
                    factor_reduced = factor.reduce(
                        [(evidence_var, evidence[evidence_var])], inplace=False
                    )
                    for var in factor_reduced.scope():
                        working_factors[var].remove((factor, origin))
                        working_factors[var].add((factor_reduced, evidence_var))
                if type(evidence[evidence_var]) != list:
                    del working_factors[evidence_var]
        return working_factors
    
    def _get_elimination_order(
        self, variables, evidence, elimination_order="minfill", show_progress=False
    ):
        """
        Deals with all elimination order parameters given to _variable_elimination method
        and returns a list of variables that are to be eliminated
        Parameters
        ----------
        elimination_order: str or list
        Returns
        -------
        list: A list of variables names in the order they need to be eliminated.
        """
        not_evidence_eliminate = []
        if evidence is not None:
            for key in evidence:
                if type(evidence[key]) != list:
                    not_evidence_eliminate.append(key)
        to_eliminate = (
            set(self.variables)
            - set(variables)
            - set(not_evidence_eliminate)
        )
        print(elimination_order)
        # Step 1: If elimination_order is a list, verify it's correct and return.
        if hasattr(elimination_order, "__iter__") and (
            not isinstance(elimination_order, str)
        ):
            if any(
                var in elimination_order
                for var in set(variables).union(
                    set(evidence.keys() if evidence else [])
                )
            ):
                raise ValueError(
                    "Elimination order contains variables which are in"
                    " variables or evidence args"
                )
            else:
                return elimination_order

        # Step 2: If elimination order is None or a Markov model, return a random order.
        elif (elimination_order is None) or (not isinstance(self.model, BayesianModel)):
            return to_eliminate

        # Step 3: If elimination order is a str, compute the order using the specified heuristic.
        elif isinstance(elimination_order, str) and isinstance(
            self.model, BayesianModel
        ):
            heuristic_dict = {
                "weightedminfill": WeightedMinFill,
                "minneighbors": MinNeighbors,
                "minweight": MinWeight,
                "minfill": MinFill,
            }
            elimination_order = heuristic_dict[elimination_order.lower()](
                self.model
            ).get_elimination_order(nodes=to_eliminate, show_progress=show_progress)
            return elimination_order
    
    def _variable_elimination(
        self,
        variables,
        operation,
        evidence=None,
        elimination_order="MinFill",
        joint=True,
        show_progress=False,
    ):
        """
        Implementation of a generalized variable elimination.

        Parameters
        ----------
        variables: list, array-like
            variables that are not to be eliminated.

        operation: str ('marginalize' | 'maximize')
            The operation to do for eliminating the variable.

        evidence: dict
            a dict key, value pair as {var: state_of_var_observed}
            None if no evidence

        elimination_order: str or list (array-like)
            If str: Heuristic to use to find the elimination order.
            If array-like: The elimination order to use.
            If None: A random elimination order is used.
        """
        # Step 1: Deal with the input arguments.
        if isinstance(variables, str):
            raise TypeError("variables must be a list of strings")
        if isinstance(evidence, str):
            raise TypeError("evidence must be a list of strings")

        # Dealing with the case when variables is not provided.
        if not variables:
            all_factors = []
            for factor_li in self.factors.values():
                all_factors.extend(factor_li)
            if joint:
                return factor_product(*set(all_factors))
            else:
                return set(all_factors)

        # Step 2: Prepare data structures to run the algorithm.
        eliminated_variables = set()
        # Get working factors and elimination order
        tic = time.time()
        working_factors = self._get_working_factors(evidence)
        toc = time.time()
        print(f"getting working factors takes {toc-tic} secs")
        elimination_order = self._get_elimination_order(
            variables, evidence, elimination_order, show_progress=show_progress
        )
        print(f"getting elimination orders takes {time.time()-toc} secs")
        # Step 3: Run variable elimination
        if show_progress:
            pbar = tqdm(elimination_order)
        else:
            pbar = elimination_order

        for var in pbar:
            tic = time.time()
            print(var)
            if show_progress:
                pbar.set_description("Eliminating: {var}".format(var=var))
            # Removing all the factors containing the variables which are
            # eliminated (as all the factors should be considered only once)
            factors = [
                factor
                for factor, _ in working_factors[var]
                if not set(factor.variables).intersection(eliminated_variables)
            ]
            phi = factor_product(*factors)
            phi = getattr(phi, operation)([var], inplace=False)
            del working_factors[var]
            for variable in phi.variables:
                working_factors[variable].add((phi, var))
            eliminated_variables.add(var)
            print(f"eliminating {var} takes {time.time()-tic} secs")
            
        # Step 4: Prepare variables to be returned.
        tic = time.time()
        final_distribution = set()
        for node in working_factors:
            for factor, origin in working_factors[node]:
                if not set(factor.variables).intersection(eliminated_variables):
                    final_distribution.add((factor, origin))
        final_distribution = [factor for factor, _ in final_distribution]
        print(final_distribution)
        print(f"the rest takes {time.time()-tic} secs")
        if joint:
            if isinstance(self.model, BayesianModel):
                return factor_product(*final_distribution).normalize(inplace=False)
            else:
                return factor_product(*final_distribution)
        else:
            query_var_factor = {}
            for query_var in variables:
                phi = factor_product(*final_distribution)
                query_var_factor[query_var] = phi.marginalize(
                    list(set(variables) - set([query_var])), inplace=False
                ).normalize(inplace=False)
            print(f"the rest takes {time.time()-tic} secs")
            return query_var_factor

    def query(
        self,
        variables,
        evidence=None,
        elimination_order="MinFill",
        joint=True,
        show_progress=False,
    ):
        """
        Parameters
        ----------
        variables: list
            list of variables for which you want to compute the probability

        evidence: dict
            a dict key, value pair as {var: state_of_var_observed}
            None if no evidence

        elimination_order: list
            order of variable eliminations (if nothing is provided) order is
            computed automatically

        joint: boolean (default: True)
            If True, returns a Joint Distribution over `variables`.
            If False, returns a dict of distributions over each of the `variables`.
        """
        common_vars = set(evidence if evidence is not None else []).intersection(
            set(variables)
        )
        if common_vars:
            raise ValueError(
                f"Can't have the same variables in both `variables` and `evidence`. Found in both: {common_vars}"
            )

        return self._variable_elimination(
            variables=variables,
            operation="marginalize",
            evidence=evidence,
            elimination_order=elimination_order,
            joint=joint,
            show_progress=show_progress,
        )

In [3]:
with open('check_points/Census_chow-liu.pkl', 'rb') as f:
    BN = pickle.load(f)
#with open('check_points/Census_junction.pkl', 'rb') as f:
 #   BN_J = pickle.load(f)

In [ ]:
def convert_to_pandas_query(query):
    query_str = ""
    for attr in query:
        query_str += attr
        val = query[attr]
        if type(val) != list:
            query_str += (" == " + str(val) + " and ")
        elif len(val) == 1:
            query_str += (" == " + str(val[0]) + " and ")
        else:
            query_str += (" in " + str(val) + " and ")
    query_str = query_str[:-5]
    return query_str

In [ ]:
len(df.query(convert_to_pandas_query(tab_query)))

In [ ]:
from time import perf_counter
def evaluate_cardinality(BN, query_path, samp, ratio=0.01):
    # read all queries
    with open(query_path) as f:
        queries = f.readlines()
    latencies = []
    q_errors = []
    for query_no, query_str in enumerate(queries):
        cardinality_true = int(query_str.split("||")[-1])
        query_str = query_str.split("||")[0]
        print(f"Predicting cardinality for query {query_no}: {query_str}")
        
        query = parse_query_single_table(query_str.strip(), BN)
        query = convert_to_pandas_query(query)
        card_start_t = perf_counter()
        cardinality_predict = len(samp.query(query))*(1/ratio)
        if cardinality_predict is None:
            continue
        card_end_t = perf_counter()
        latency_ms = (card_end_t - card_start_t) * 1000
        if cardinality_predict == 0 and cardinality_true == 0:
            q_error = 1.0
        elif cardinality_predict == 0:
            cardinality_predict = 1
        elif cardinality_true == 0:
            cardinality_true = 1

        q_error = max(cardinality_predict / cardinality_true, cardinality_true / cardinality_predict)
        print(f"latency: {latency_ms} and error: {q_error}")
        latencies.append(latency_ms)
        q_errors.append(q_error)
    return latencies, q_errors

In [ ]:
query_path = "/home/ziniu.wzn/Census/cardinality/query_one_side.sql"
latencies, q_errors = evaluate_cardinality(BN_J, query_path, df.sample(n=int(len(df)*0.001)), 0.001)

In [ ]:
print(np.mean(latencies))
for i in [50,90,95,99,100]:
    print(np.percentile(q_errors, i))

In [4]:
query = "SELECT COUNT(*) FROM climate WHERE iAvail = 0 AND iClass = 0 AND iKorean >= 0 AND iKorean <= 1 AND iMay75880 = 0 AND iRagechld >= 0 AND iRagechld <= 4 AND iRrelchld = 0"
query = parse_query_single_table(query.strip(), BN)
print(query)
tic = time.time()
BN.query(query)
print(time.time() - tic)

{'iAvail': [0], 'iClass': [0], 'iKorean': [0, 1], 'iMay75880': [0], 'iRagechld': [4, 2, 0, 3, 1], 'iRrelchld': [0]}
getting working factors takes 0.1161050796508789 secs
MinFill
getting elimination orders takes 0.011570930480957031 secs
iMobility
eliminating iMobility takes 0.00035572052001953125 secs
iClass
eliminating iClass takes 0.0008523464202880859 secs
dIncome5
eliminating dIncome5 takes 0.00026106834411621094 secs
iDisabl2
eliminating iDisabl2 takes 0.0007538795471191406 secs
iSchool
eliminating iSchool takes 0.00026917457580566406 secs
iMay75880
eliminating iMay75880 takes 0.0003838539123535156 secs
iWorklwk
eliminating iWorklwk takes 0.00026702880859375 secs
dIncome4
eliminating dIncome4 takes 0.00026702880859375 secs
iRownchld
eliminating iRownchld takes 0.0004017353057861328 secs
iFeb55
eliminating iFeb55 takes 0.0002951622009277344 secs
dIncome2
eliminating dIncome2 takes 0.0002899169921875 secs
iRiders
eliminating iRiders takes 0.00029206275939941406 secs
iMeans
eliminati

eliminating dAge takes 2.1520090103149414 secs
dDepart
eliminating dDepart takes 0.2741830348968506 secs
dIncome7
eliminating dIncome7 takes 0.13897085189819336 secs
dAncstry2
eliminating dAncstry2 takes 0.018959760665893555 secs
iOthrserv
eliminating iOthrserv takes 0.0070819854736328125 secs
dOccup
eliminating dOccup takes 0.0010251998901367188 secs
dHour89
eliminating dHour89 takes 0.0004889965057373047 secs
[<DiscreteFactor representing phi(iRagechld:5) at 0x104879650>, <DiscreteFactor representing phi(iRagechld:5) at 0x1a28c33490>]
the rest takes 0.0004668235778808594 secs
getting working factors takes 0.0956113338470459 secs
MinFill
getting elimination orders takes 0.013314008712768555 secs
iMobility
eliminating iMobility takes 0.00037384033203125 secs
iClass
eliminating iClass takes 0.0009410381317138672 secs
dIncome5
eliminating dIncome5 takes 0.0002999305725097656 secs
iDisabl2
eliminating iDisabl2 takes 0.0008680820465087891 secs
iSchool
eliminating iSchool takes 0.0002980232

eliminating dAge takes 2.347504138946533 secs
dDepart
eliminating dDepart takes 0.2798750400543213 secs
dIncome7
eliminating dIncome7 takes 0.11316299438476562 secs
dAncstry2
eliminating dAncstry2 takes 0.02406001091003418 secs
iOthrserv
eliminating iOthrserv takes 0.011209964752197266 secs
iRagechld
eliminating iRagechld takes 0.002341032028198242 secs
dOccup
eliminating dOccup takes 0.0030689239501953125 secs
dHour89
eliminating dHour89 takes 0.0017910003662109375 secs
[<DiscreteFactor representing phi(iClass:10) at 0x1a28bf80d0>, <DiscreteFactor representing phi(iClass:10) at 0x1a28c462d0>]
the rest takes 0.0010488033294677734 secs
getting working factors takes 0.1262679100036621 secs
MinFill
getting elimination orders takes 0.02306199073791504 secs
iMobility
eliminating iMobility takes 0.0008800029754638672 secs
iClass
eliminating iClass takes 0.0017561912536621094 secs
dIncome5
eliminating dIncome5 takes 0.0007622241973876953 secs
iDisabl2
eliminating iDisabl2 takes 0.001750946044

eliminating dHours takes 0.548792839050293 secs
dIncome1
eliminating dIncome1 takes 0.0007512569427490234 secs
iKorean
eliminating iKorean takes 0.0005800724029541016 secs
iLang1
eliminating iLang1 takes 0.004741191864013672 secs
dPOB
eliminating dPOB takes 0.0005731582641601562 secs
dAncstry1
eliminating dAncstry1 takes 0.005207061767578125 secs
iRemplpar
eliminating iRemplpar takes 0.0005311965942382812 secs
iYearsch
eliminating iYearsch takes 0.0024640560150146484 secs
dAge
eliminating dAge takes 7.761683940887451 secs
dDepart
eliminating dDepart takes 0.946476936340332 secs
dIncome7
eliminating dIncome7 takes 0.3648960590362549 secs
dAncstry2
eliminating dAncstry2 takes 0.041829824447631836 secs
iOthrserv
eliminating iOthrserv takes 0.0004467964172363281 secs
iRvetserv
eliminating iRvetserv takes 0.020999908447265625 secs
iRagechld
eliminating iRagechld takes 0.00401616096496582 secs
dOccup
eliminating dOccup takes 0.0009779930114746094 secs
dHour89
eliminating dHour89 takes 0.0005

In [5]:
infer_machine = VariableElimination(BN.model)

In [6]:
infer_machine.query(['iAvail'], {'iClass': [0], 'iKorean': [0, 1], 'iMay75880': [0], 'iRagechld': [4, 2, 0, 3, 1], 'iRrelchld': [0]})


getting working factors takes 0.12086606025695801 secs
MinFill
getting elimination orders takes 0.013921737670898438 secs
iMobility
eliminating iMobility takes 0.00039887428283691406 secs
iClass
eliminating iClass takes 0.001043081283569336 secs
dIncome5
eliminating dIncome5 takes 0.0003027915954589844 secs
iDisabl2
eliminating iDisabl2 takes 0.0008020401000976562 secs
iSchool
eliminating iSchool takes 0.0002779960632324219 secs
iMay75880
eliminating iMay75880 takes 0.0002789497375488281 secs
iRrelchld
eliminating iRrelchld takes 0.0021820068359375 secs
iWorklwk
eliminating iWorklwk takes 0.0004322528839111328 secs
dIncome4
eliminating dIncome4 takes 0.0002808570861816406 secs
iRownchld
eliminating iRownchld takes 0.00041294097900390625 secs
iFeb55
eliminating iFeb55 takes 0.0002970695495605469 secs
dIncome2
eliminating dIncome2 takes 0.0003371238708496094 secs
iRiders
eliminating iRiders takes 0.0003561973571777344 secs
iMeans
eliminating iMeans takes 0.0011410713195800781 secs
iImmig

<DiscreteFactor representing phi(iAvail:5) at 0x1a28c3dd90>